In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
# local is master, embedded inside same jvm
# local -1 parallism - good for debugging, 1 task shall at a time
# local[2] - 2 parallisms , 2 task4 can run at same time
# local[4] - 4 parallisms , 4 task4 can run at same time
# local[*] -  number of parallism is based on number of cores in the system
sc = SparkContext("local[8]", "LocalMaster")


22/03/02 22:35:37 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/03/02 22:35:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/02 22:35:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# these are from configuration, based on number of parallel tasks
print("  defaultParallelism ", sc.defaultParallelism)
print("  defaultMinPartitions ", sc.defaultMinPartitions)

rdd = sc.parallelize(range(1, 10))
rdd = rdd.repartition(500)
print(rdd.getNumPartitions()) # sc.defaultParallelism
rdd.collect()

  defaultParallelism  8
  defaultMinPartitions  2
500


[8, 9, 5, 1, 4, 7, 2, 6, 3]

In [3]:
from pyspark import StorageLevel

In [13]:
# how to remove special chars
# we will all ascii
def to_ascii(text):
    import re
    output = re.sub(r"[^a-zA-Z0-9 ]","",text)
    #print(output)
    return output

text = "prince, don't ;welcome"
to_ascii(text)

'prince dont welcome'

In [28]:
wordCountRdd = sc.textFile("hdfs://localhost:9000/book-war-and-peace.txt")\
                 .map (lambda line: to_ascii(line))\
                  .map (lambda line: line.strip().lower())\
                 .map (lambda line: line.split(" "))\
                 .flatMap(lambda elements: elements)\
                 .filter (lambda word: word != "")\
                 .map (lambda word: (word, 1))\
                 .reduceByKey(lambda acc, value: acc + value)


In [29]:
# wordCountRdd.cache() # , call persit internally with MEMORY_ONLY

wordCountRdd.persist(StorageLevel.MEMORY_AND_DISK) # we have many options
#wordCountRdd.persist(StorageLevel.DISK_ONLY) # we have many options

wordCountRdd.take(10)

[('chapter', 367),
 ('i', 4089),
 ('well', 741),
 ('prince', 1886),
 ('so', 1852),
 ('genoa', 3),
 ('and', 22082),
 ('lucca', 2),
 ('are', 1259),
 ('now', 1305)]

In [33]:
# sort data by value, in ascending order
# least used words

sortedRddAscending = wordCountRdd.sortBy(lambda kv: kv[1])

sortedRddAscending.take(10)

[('infamies', 1),
 ('yousit', 1),
 ('elite', 1),
 ('scarletliveried', 1),
 ('10annette', 1),
 ('grandfathers', 1),
 ('canceled', 1),
 ('woundup', 1),
 ('tease', 1),
 ('novosiltsevs', 1)]

In [34]:
# sortBy values but decending order
# most used words
print(wordCountRdd.getNumPartitions())
sortedRddDecending = wordCountRdd.sortBy(lambda kv: kv[1], ascending=False)
sortedRddDecending.take(10)

1


[('the', 34395),
 ('and', 22082),
 ('to', 16636),
 ('of', 14872),
 ('a', 10464),
 ('he', 9807),
 ('in', 8744),
 ('his', 7967),
 ('that', 7798),
 ('was', 7328)]

In [32]:
wordCountRdd.saveAsTextFile("hdfs://localhost:9000/war-and-peace-12")

In [35]:
sortedRddDecending.saveAsTextFile("hdfs://localhost:9000/war-and-peace-sorted-desc2")

In [36]:
sortedRddAscending.saveAsTextFile("hdfs://localhost:9000/war-and-peace-sorted-asc2")